In [233]:
#init libraries + data
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import math

#data = pd.read_csv('sample_data/mnist_train_small.csv')
data = pd.read_csv('train.csv')

data = np.array(data)
m, n = data.shape
np.random.shuffle(data)

data_dev = data[0:1000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:n]

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 256

In [234]:
#inits weights and biases + activation functions
w_1 = np.random.rand(10, 784)
w_2 = np.random.rand(10, 10)
b_1 = np.random.rand(10)
#w_1 = np.zeros((10,784))
#w_2 = np.zeros((10,10))
#b_1 = np.zeros(10)

def relu(x):
    return (x>0) * x
    
def drelu(x):
    return x > 0

def sigmoid(x):
    return 1/(1 + (math.e)**(-x))

def dsigmoid(x):
    return sigmoid(x) * (1-sigmoid(x))

In [235]:
def feedforward(activation, w, b, a):
  z_out = np.matmul(w,a)
  z_out += b.reshape(b.size)
  n = z_out.size
  a_out = np.copy(z_out)
  if activation == "sigmoid":
    for i in range(n):
      a_out[i] = sigmoid(z_out[i])
  elif activation == "relu":
    for i in range(n):
      a_out[i] = relu(z_out[i])
  elif activation == "none":
    return z_out,a_out
  else:
    return "Unrecognized activation function"
  return z_out,a_out
      
def cost(a, i):
  temp = np.zeros(10)
  temp[int(Y_train[i])] = 1
  return np.sum((a-temp)**2)
  
def backprop(activation,z,a,w,dz):
  dw = np.matmul(dz.reshape((dz.size,1)),a.reshape((1,a.size)))
  #print(dw)
  dz_out = np.matmul(dz.reshape((1,dz.size)),w)
  n = dz_out.size
  if activation == "sigmoid":
    for x in range(n):
      dz_out[0][x] = dz_out[0][x] * dsigmoid(z[x])
  elif activation == "relu":
    for x in range(n):
      dz_out[0][x] = dz_out[0][x] * drelu(z[x])
  else:
    return dz_out.reshape(n),dw
  return dz_out.reshape(n),dw

a = np.ones(10)
a[5:10] = 2
w = np.ones((10,10))
w[5:10,5:10] = 2
backprop("relu",a,a,w,np.ones(10))

(array([10., 10., 10., 10., 10., 15., 15., 15., 15., 15.]),
 array([[1., 1., 1., 1., 1., 2., 2., 2., 2., 2.],
        [1., 1., 1., 1., 1., 2., 2., 2., 2., 2.],
        [1., 1., 1., 1., 1., 2., 2., 2., 2., 2.],
        [1., 1., 1., 1., 1., 2., 2., 2., 2., 2.],
        [1., 1., 1., 1., 1., 2., 2., 2., 2., 2.],
        [1., 1., 1., 1., 1., 2., 2., 2., 2., 2.],
        [1., 1., 1., 1., 1., 2., 2., 2., 2., 2.],
        [1., 1., 1., 1., 1., 2., 2., 2., 2., 2.],
        [1., 1., 1., 1., 1., 2., 2., 2., 2., 2.],
        [1., 1., 1., 1., 1., 2., 2., 2., 2., 2.]]))

In [236]:
def relu_epoch(alpha,w_1, w_2, b_1):
  dz_2 = np.ones(10)
  for i in range(2):
    z_1,a_1 = feedforward("relu",w_1,b_1,X_train[:,i])
    z_2,a_2 = feedforward("relu",w_2,np.zeros(10),a_1) 
    #solution array
    y_2 = np.zeros(10) 
    y_2[int(Y_train[i])] = 1
    #dw_2
    da_2 = a_2-y_2
    for x in range(10):
      dz_2[x] = da_2[x] * drelu(z_2[x])
    dz_1, dw_2 = backprop("relu",z_1,a_1,w_2,dz_2)

    dw_1 = backprop("none",X_train[:,i],X_train[:,i],w_1,dz_1)[1]
    #print(dw_2)
    w_1 =w_1- alpha * dw_1
    w_2 =w_2- alpha * dw_2
    b_1 =b_1- alpha * dz_1 
    #print(w_1,w_2,b_1)
  return w_1,w_2,z_1

In [237]:
#print(np.matmul(w_1,X_train[:,1]))
#print(feedforward("relu",w_1,np.zeros(10),X_train[:,1])[1])
#print(feedforward("relu",w_2,np.zeros(10),feedforward("relu",w_1,np.zeros(10),X_train[:,1])[1]))
for i in range(10):
    w_1, w_2, b_1 = relu_epoch(0.0000000000000000001, w_1, w_2, b_1)
    z_1,a_1 = feedforward("relu",w_1,b_1,X_train[:,1])
    z_2,a_2 = feedforward("relu",w_2,np.zeros(10),a_1)
    print(z_1,a_1)
    #print(cost(a_2,2))

[162.07872109 157.61347838 146.00346639 138.80471943 178.82357597
 159.60444419 155.99243032 160.88558843 155.73337904 162.06478187] [162.07872109 157.61347838 146.00346639 138.80471943 178.82357597
 159.60444419 155.99243032 160.88558843 155.73337904 162.06478187]
[242.8041427  236.18889161 218.66551614 208.08680618 267.86501219
 239.13114944 233.63459871 241.01597753 233.48752792 242.65736374] [242.8041427  236.18889161 218.66551614 208.08680618 267.86501219
 239.13114944 233.63459871 241.01597753 233.48752792 242.65736374]
[323.52956431 314.76430483 291.32756588 277.36889292 356.9064484
 318.65785468 311.27676709 321.14636663 311.2416768  323.24994561] [323.52956431 314.76430483 291.32756588 277.36889292 356.9064484
 318.65785468 311.27676709 321.14636663 311.2416768  323.24994561]
[404.25498591 393.33971806 363.98961563 346.65097966 445.94788461
 398.18455992 388.91893547 401.27675572 388.99582567 403.84252748] [404.25498591 393.33971806 363.98961563 346.65097966 445.94788461
 398.

In [238]:
#accuracy
def accuracy(w_1,w_2,b_1):
    n = 0
    for i in range(18999):
        a_1 = feedforward("sigmoid",w_1,b_1,X_train[:,i])[1]
        a_2 = feedforward("sigmoid",w_2,np.zeros(10),a_1)[1] 
        max = 0
        index = 0
        for x in range(10):
            if a_2[x] > max:
                max = a_2[x]
                index = x
        if Y_train[i] == x:
            n += 1
    return n / 18999
for i in range(10):
    a_1 = feedforward("sigmoid",w_1,b_1,X_train[:,i])[1]
    a_2 = feedforward("sigmoid",w_2,np.zeros(10),a_1)[1] 
    print(a_2,Y_train[i])

[0.98992897 0.98724408 0.99821227 0.9918403  0.99303204 0.987616
 0.98077665 0.99645377 0.99124901 0.97303339] 5
[0.98992897 0.98724408 0.99821227 0.9918403  0.99303204 0.987616
 0.98077665 0.99645377 0.99124901 0.97303339] 2
[0.98992897 0.98724408 0.99821227 0.9918403  0.99303204 0.987616
 0.98077665 0.99645377 0.99124901 0.97303339] 8
[0.98992897 0.98724408 0.99821227 0.9918403  0.99303204 0.987616
 0.98077665 0.99645377 0.99124901 0.97303339] 6
[0.98992897 0.98724408 0.99821227 0.9918403  0.99303204 0.987616
 0.98077665 0.99645377 0.99124901 0.97303339] 7
[0.98992897 0.98724408 0.99821227 0.9918403  0.99303204 0.987616
 0.98077665 0.99645377 0.99124901 0.97303339] 8
[0.98992897 0.98724408 0.99821227 0.9918403  0.99303204 0.987616
 0.98077665 0.99645377 0.99124901 0.97303339] 5
[0.98992897 0.98724408 0.99821227 0.9918403  0.99303204 0.987616
 0.98077665 0.99645377 0.99124901 0.97303339] 6
[0.98992897 0.98724408 0.99821227 0.9918403  0.99303204 0.987616
 0.98077665 0.99645377 0.991249